In [25]:
import pandas as pd

# ─────────────────────────────────────────────────────────────
# 1. PARAMETERS & INPUTS
# ─────────────────────────────────────────────────────────────
installed_capacity = {
    2020: 33730.56,
    2021: 37464.64,
    2022: 49346.71,
    2023: 63302.52,
    2024: 73318.49,
    2030: 280000.00
}

poa_csv     = r"D:\IIT BOMBAY\May 2025\Final_8_Nodes\POA.csv"
demand_xlsx = r"D:\IIT BOMBAY\May 2025\Final_8_Nodes\Demand 2020_24.xlsx"
output_xlsx = r"D:\IIT BOMBAY\May 2025\Final_8_Nodes\combined_data_with_storage.xlsx"

# STORAGE — apply to **2030 only**:
storage_power_mw    = 97350             # full-power rating in MW
energy_capacity_mwh = storage_power_mw * 6   # 6 h at full power
discharge_rate_mw   = storage_power_mw / 2    # half-rate so it lasts 12 h

# ─────────────────────────────────────────────────────────────
# 2. READ & PREPARE
# ─────────────────────────────────────────────────────────────
df = pd.read_csv(poa_csv)
df['datetime']       = pd.to_datetime(df['timestamp'])
df['year']           = df['datetime'].dt.year
df['month_day']      = df['datetime'].dt.strftime('%m-%d')
df['hour']           = df['datetime'].dt.hour

# plane-of-array → MW
df['solar_generation'] = df.apply(
    lambda r: installed_capacity[r.year] * r.poa / 1000,
    axis=1
)

# demand & original net_load
d = pd.read_excel(demand_xlsx)
if len(df) != len(d):
    raise ValueError("Row count mismatch between POA.csv and Demand.xlsx")
df['demand']   = d['Hourly Demand Met (in MW)']
df['net_load'] = df['demand'] - df['solar_generation']

# initialize flattened same as original    
df['net_load_flattened'] = df['net_load'].copy()

# ─────────────────────────────────────────────────────────────
# 3. APPLY STORAGE LOGIC FOR 2030 (FORCED CHARGE SCHEDULE)
# ─────────────────────────────────────────────────────────────
mask2030 = df['year'] == 2030

for day in df.loc[mask2030, 'month_day'].unique():
    sub = df[mask2030 & (df['month_day'] == day)].sort_values('hour')
    storage_lvl = 0.0   # MWh on board

    for idx, row in sub.iterrows():
        nl    = row['net_load']
        solar = row['solar_generation']
        h     = row['hour']

        if 7 <= h < 19:
            # SUN HOURS: force-charge at up to storage_power_mw from solar
            charge = min(
                storage_power_mw,         # your 75 180 MW max-power
                solar,                    # you can only take from whatever solar gives
                energy_capacity_mwh - storage_lvl
            )
            storage_lvl += charge
            nl += charge            # bump net-load by the storage draw

        else:
            # DARK HOURS: discharge at half-power up to what's stored
            discharge = min(
                discharge_rate_mw,       # 37 590 MW discharge power
                storage_lvl
            )
            storage_lvl -= discharge
            nl -= discharge          # shave net-load by the storage supply

        df.at[idx, 'net_load_flattened'] = nl

# ─────────────────────────────────────────────────────────────
# 4. SAVE
# ─────────────────────────────────────────────────────────────
df.to_excel(output_xlsx, index=False)
print("Saved combined data with storage flattening →", output_xlsx)



C:\Users\sumit\AppData\Local\Temp\ipykernel_1508\128233106.py:28: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['datetime']       = pd.to_datetime(df['timestamp'])


Saved combined data with storage flattening → D:\IIT BOMBAY\May 2025\Final_8_Nodes\combined_data_with_storage.xlsx


In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# ───────────────────────────────────────────────────────────────────
# 1) LOAD THE COMBINED DATA AND REBUILD hour / hour_shifted
# ───────────────────────────────────────────────────────────────────
df = pd.read_excel(
    r"D:\IIT BOMBAY\May 2025\Final_8_Nodes\combined_data_with_storage.xlsx"
)

# If for some reason you never had these in the Excel, recreate them:
if 'hour' not in df.columns or 'hour_shifted' not in df.columns:
    df['datetime']    = pd.to_datetime(df['timestamp'])
    df['hour']        = df['datetime'].dt.hour
    df['hour_shifted']= (df['hour'] - 23) % 24

# ───────────────────────────────────────────────────────────────────
# 2) SETUP OUTPUT FOLDER
# ───────────────────────────────────────────────────────────────────
output_folder = r"D:\IIT BOMBAY\May 2025\Final_8_Nodes\dc_Jan_upto_2030"
os.makedirs(output_folder, exist_ok=True)

# ───────────────────────────────────────────────────────────────────
# 3) LOOP OVER DAYS AND YEARS TO PLOT
# ───────────────────────────────────────────────────────────────────
for day in sorted(df['month_day'].unique()):
    today = df[df['month_day']==day]
    plt.figure(figsize=(10,5))

    # Plot all original duck curves
    for yr in sorted(today['year'].unique()):
        yd = today[today['year']==yr].sort_values('hour_shifted')
        if yd.empty:
            continue

        # append first point at 24 to close the loop
        first = yd.iloc[0].copy()
        first['hour_shifted'] = 24
        yd = pd.concat([yd, pd.DataFrame([first])], ignore_index=True)

        style = '--' if yr==2030 else '-'  
        plt.plot(
            yd['hour_shifted'],
            yd['net_load'],
            marker='o',
            linestyle=style,
            label=f"{yr} Duck"
        )

    # Overlay 2030’s flattened curve
    flat = today[today['year']==2030].sort_values('hour_shifted')
    if not flat.empty:
        first = flat.iloc[0].copy()
        first['hour_shifted'] = 24
        flat = pd.concat([flat, pd.DataFrame([first])], ignore_index=True)
        plt.plot(
            flat['hour_shifted'],
            flat['net_load_flattened'],
            linewidth=2.5,
            label="2030 Flattened"
        )

    # X-axis formatting
    xt = list(range(0,25,2))
    xl = [
        "23:00" if x in (0,24)
        else f"{(x-1)%24:02d}:00"
        for x in xt
    ]
    plt.xticks(xt, xl)
    plt.xlabel("Hour of the Day")
    plt.ylabel("Net Load (MW)")
    plt.title(f"Duck Curve vs. Flattened (Storage) — {day}")
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.legend(title="Year / Curve", fontsize=9)

    fn = os.path.join(output_folder, f"duck_curve_flat_{day}.png")
    plt.savefig(fn, bbox_inches='tight')
    plt.close()

print("All plots saved to", output_folder)

C:\Users\sumit\AppData\Local\Temp\ipykernel_1508\1981383587.py:14: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['datetime']    = pd.to_datetime(df['timestamp'])


All plots saved to D:\IIT BOMBAY\May 2025\Final_8_Nodes\dc_Jan_upto_2030
